<a href="https://colab.research.google.com/github/sahooabhipsa10/MediQuery/blob/main/MediQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install SpeechRecognition


In [ ]:
!pip install --upgrade pip

In [ ]:
!apt-get install portaudio19-dev


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 19 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 0s (462 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 120874 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.

In [ ]:
!pip install pyaudio


In [ ]:
import speech_recognition as sr

In [ ]:
pip install pyttsx3

In [ ]:
!pip install ffmpeg-python

In [ ]:
# Import necessary libraries
from base64 import b64decode
from google.colab import output
from IPython.display import Javascript
import io
import tempfile

# JavaScript code for recording audio
RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async () => {
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

# Function to record audio and return it as a base64 encoded string
def record_audio(sec=30):
    print("Speak Now...")
    display(Javascript(RECORD))
    sec += 1

    audio_data = output.eval_js('record(%d)' % (sec * 1000))
    print("Done Recording!")
    return audio_data

# Call the function to record audio
audio_data = record_audio()

# In a separate code cell, use SpeechRecognition to convert the audio to text
import speech_recognition as sr
from pydub import AudioSegment

# Initialize the recognizer
recognizer = sr.Recognizer()

# Function to convert audio data to text
def audio_to_text(audio_data):
    audio_bytes = b64decode(audio_data.split(',')[1])

    # Save the audio as a temporary WAV file
    with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as temp_audio_file:
        temp_audio_file.write(audio_bytes)
        temp_audio_file.seek(0)
        audio_file_name = temp_audio_file.name

    # Convert the audio to WAV format
    audio = AudioSegment.from_file(audio_file_name, format="webm")
    audio.export(audio_file_name, format="wav")

    with sr.AudioFile(audio_file_name) as source:
        audio = recognizer.record(source)

    try:
        text = recognizer.recognize_google(audio, language="eng-IN")
        return text
    except sr.UnknownValueError:
        return "Speech recognition could not understand the audio"
    except sr.RequestError as e:
        return f"Could not request results from Google Speech Recognition service; {e}"

# Convert the audio data to text
recognized_text = audio_to_text(audio_data)
print("Recognized Text:")
print(recognized_text)



Speak Now...


<IPython.core.display.Javascript object>

Done Recording!
Recognized Text:
the patient has fever the symptoms are cold cough and high temperature they have to take paracetamol two times a day after each meal they should take rest sleep they should have eight hours of sleep and take electrolytes


In [ ]:
!pip install pydub pyaudio SpeechRecognition


In [ ]:
 !pip install pydub


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.1/123.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 21.1 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import BertForQuestionAnswering, BertTokenizer

# Load the pre-trained BERT model and tokenizer with return_dict=False
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
model = BertForQuestionAnswering.from_pretrained(model_name, return_dict=False)
tokenizer = BertTokenizer.from_pretrained(model_name)

def answer_question(question, context):
    # Tokenize the input text
    input_ids = tokenizer.encode(question, context)

    # Perform question answering
    start_scores, end_scores = model(torch.tensor([input_ids]))

    # Find the answer span with the highest probability
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Convert answer span token IDs back to text
    answer = tokenizer.decode(input_ids[answer_start:answer_end+1])

    return answer

# Get user's input question
context = recognized_text
question = input("Ask a medical question: ")

answer = answer_question(question, context)
print("Answer:", answer)


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Ask a medical question: how many times should I take paracetamol?
Answer: two times a day
